## Angular Distribution Analysis Using Dump DY

 To see the points in the histograms interactively [CLICK HERE](https://nbviewer.org/github/forhadnmsu/DumpAna/blob/main/dumpAna.ipynb)

Even after subtracting the combinatorial background (signal =unmixed events - mixed events), we deal with a large number of backgrounds. By finding the optimum event selections, we can remove the unphysical dimuons from the signal+other Background events. To take care of that, we can compare Monte Carlo distributions and find optimum selections. This is called signal to background significance $\frac{S}{\sqrt{S+B}}$. The details study can be found in the [docdb:10221](https://seaquest-docdb.fnal.gov/cgi-bin/sso/ShowDocument?docid=10221). From the study, the track and dimuon event selections have been added to the ["cutDUMPDY.h"](https://github.com/forhadnmsu/DumpAna/blob/main/cutDUMPDY.h). The reconstructed events from the SeaQuest roadset-67 dataset and the reconstructed data from the Monte Carlo will get affected by the same even selections. All the seelctions will pass to the [RDataFrame Filter](https://root.cern/doc/master/classROOT_1_1RDataFrame.html). In the Filter, we have a lot selections that are added as && condition, and they are executed in order and the first one returning false causes the event to be discarded.

In [1]:
#include "TH1.h"
#include "TH2.h"
#include "TH3.h"
#include "TFile.h"
#include "TTree.h"
#include "TROOT.h"
#include "TStyle.h"
#include "TCanvas.h"
#include "TCut.h"
#include "TLegend.h"
#include "TLine.h"
#include "TLatex.h"
#include "TMarker.h"
#include "ana_const.h"
#include "cutDUMPDY.h"

The static function [SetDefaultSumw2()](https://root.cern/doc/master/classTH1.html#a8cd3b44217c8fec1d74046ed2ac153c2) is called in the beginning since we are going to deal with the weighted Histograms. Therefore the Sumw2() function is automatically called whenever any histograms is created.

In [2]:
%jsroot on
ROOT::EnableImplicitMT();
TH1::SetDefaultSumw2();
TH2::SetDefaultSumw2();
gStyle->SetPalette(1);
gStyle->SetPaintTextFormat("5.4f");
gROOT->ForceStyle();
gStyle->SetOptStat(0);
gStyle->SetOptFit(1111);
ROOT::RDataFrame dmyDf_data("result", "merged_RS67_3087Unmixall_target.root");//All Target combined "unmixed" reconstructed toadset-67 events 
ROOT::RDataFrame dmyDf_mix("result_mix", "merged_RS67_3087Mixall_target.root");//All Target combined "mixed" reconstructed toadset-67 events 
TH1D* h_pt = new TH1D("h_pt","h_pt", nBinpt, ptMin, ptMax);  // for pT Binning
TH1D* h_x2 = new TH1D("h_x2","h_x2", nBinx2, x2Min, x2Max);  // for xT binning
TFile *myfile = TFile::Open("acc_factor.root","read");
TF2* fit2D = new TF2("fit2D", "[0] * ( 1 + [1]*y*y + 2*[2]*sqrt(1-y*y)*y*cos(x) + [3]*(1-y*y)*cos(2*x)/2.) ", -M_PI, M_PI, -0.5, 0.5);
fit2D->SetParameters(1,1,0,0);
fit2D->SetParNames("A", "#lambda","#mu","#nu");

Using the function ResetEmptyBins, we have ignored the low bin content, therefore, low accepted bin combinations can be ignored in the chi2-fitting. 

In [3]:
TH2D * ResetEmptyBins(
    const char* hname,
    TH2D* h_phi_costh )
{   
    int nbinX = h_phi_costh->GetNbinsX();
    int nbinY = h_phi_costh->GetNbinsY();
    TH2D *h = (TH2D*)h_phi_costh->Clone();
    for (int ii=1; ii<= nbinX; ii++){
        for (int jj=1; jj<= nbinY; jj++){

            if(h_phi_costh->GetBinContent(ii,jj) <=5){
                h->SetBinContent(ii,jj,0);
                h->SetBinError(ii,jj,0);
            };  
        }   
    }   
    return h;
}

Angular distribution coefficients in the following conditions: 

1. Data: All Mixed target data.
2. GMC : LD2 random hit embedded Drell--Yan [docdb:9886](https://seaquest-docdb.fnal.gov/cgi-bin/sso/ShowDocument?docid=9886) 
4. 4 Dimuon pT bins in the range (0 to 2.0 GeV)
       

In [4]:
TCanvas *c[5];
TFile *myfile = TFile::Open("acc_factor.root","read");
TF2* fit2D = new TF2("fit2D", "[0] * ( 1 + [1]*y*y + 2*[2]*sqrt(1-y*y)*y*cos(x) + [3]*(1-y*y)*cos(2*x)/2.) ", -M_PI, M_PI, costhMin, costhMax);
fit2D->SetParameters(1,1,0,0);
fit2D->SetParNames("A", "#lambda","#mu","#nu");
int can =0;
for (int i_xT = 1; i_xT <=nBinx2 ; i_xT++) {
    for (int i_pt = 1; i_pt <=nBinpt ; i_pt++) {
        c[i_pt] = new TCanvas(Form("c%i",i_pt),"",800,1500);
        c[i_pt]->Divide(2,6);
        for (int i_D1 = 1; i_D1 <=nBinD1 ; i_D1++) {
            Double_t xBinMinPt = h_pt->GetXaxis()->GetBinLowEdge(i_pt);
            Double_t xBinMaxPt = h_pt->GetXaxis()->GetBinUpEdge(i_pt);
            Double_t xBinMinx2 = h_x2->GetXaxis()->GetBinLowEdge(i_xT);
            Double_t xBinMax2 = h_x2->GetXaxis()->GetBinUpEdge(i_xT);
            //RDATAFRAME
            int D1_low = range_D1*(i_D1-1);
            int D1_high = range_D1*i_D1;
            string  ptCutTrue = Form("true_pT>%4.10f&&true_pT<%4.10f&&",xBinMinPt,xBinMaxPt);
            string  occu =Form ("D1>%i&&D1<%i&&",D1_low,D1_high);
            //string massCut = "true_mass>5.0 && true_mass<8.0";
            string massCut = Form("true_mass>%f && true_mass<%f",MassMin, MassMax);
            string pTBins= Form("pT>%4.10f&&pT<%4.10f&&",xBinMinPt,xBinMaxPt);
            string xTBins=  Form("xT>%4.10f&&xT<%4.10f&&",xBinMinx2,xBinMax2);
            //string massRange=" mass>5.0 && mass<8.0&&";
            string massRange=Form(" mass>%f && mass<%f&&",MassMin, MassMax);
            string final_cut_filter=occu+pTBins+xTBins+massRange+dumpCutRD; // data filter 
            cout << "cut in data: "<< final_cut_filter << endl;

            auto hist2d_unmix = dmyDf_data.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_unmix", "phi_theta_unmix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax}, "phi_", "costh_");
            auto hist2d_mix = dmyDf_mix.Filter(final_cut_filter).Define("phi_","phi").Define("costh_","costh").Histo2D({"phi_theta_mix", "phi_theta_mix", nBinPhi,phiMin, phiMax, nBinCosth, costhMin, costhMax},"phi_", "costh_");
            TH2D* acc_factor = (TH2D*)myfile->Get(Form("h2_phi_costh_ixT_%i_ipt_%i_iD1_%i",i_xT ,i_pt, i_D1));
            hist2d_unmix->Add(hist2d_mix.GetPtr(),-1);
            hist2d_unmix->SetName(Form("phi_theta_ipT%i_i_D1_%i",i_pt,i_D1));
            TH2D*h2_phi_costh = ResetEmptyBins("h2_phi_costh",hist2d_unmix.GetPtr());
            //TH2D*h2_phi_costh = (TH2D*)hist2d_unmix->Clone();
            int analized_data = h2_phi_costh->Integral();
            //out_File2 << setprecision(4) <<Form("$%i<D1<%i$, $%4.2f<pT<%4.2f$",D1_low,D1_high,xBinMinPt,xBinMaxPt)<<",";
            can++;
            c[i_pt]->cd(can);
            h2_phi_costh->Draw("colz text");
            h2_phi_costh->SetTitle(Form("(Unmix -Mix) Events in  pT_{low}=%4.2f and pT_{high}=%4.2f  and D1_{low}=%i&&D1_{high}=%i ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt, D1_low,D1_high));
            can++;
            c[i_pt]->cd(can);
            TH2*acc_corrected_h2d = (TH2D*)h2_phi_costh->Clone();
            //TH2D*acc_corrected_h2d = ResetEmptyBins("acc_corrected_h2d",h2_phi_costh);    
            acc_corrected_h2d->Divide(acc_factor);
            /*
                                   bool fitOK = false;
                                   while( ! fitOK ){
                                   acc_corrected_h2d->Fit("fit2D","M");
                                   double tempChisq = fit2D->GetChisquare() / fit2D->GetNDF();
                                   fitOK = tempChisq < 16.0 ? true : false;
                                   }
                                 */
            acc_corrected_h2d->Fit("fit2D");
            std::setprecision(4);
            cout << fit2D->GetChisquare() / fit2D->GetNDF() << endl;
            acc_corrected_h2d->SetTitle(Form("Acc. corrected in pT_{low}=%4.2f and pT_{high}=%4.2f  and D1_{low}=%i&&D1_{high}=%i ;#phi_{CS};cos#theta_{CS}",xBinMinPt,xBinMaxPt, D1_low,D1_high));
            acc_corrected_h2d->Draw("colz text");
            if(can==12){
                c[i_pt]->SaveAs(Form("plot/acc_corrected_iPT_%i.pdf",i_pt));
                c[i_pt]->Draw();
                can=0;
            }
            delete acc_factor;
        }
    }
}


cut in data: D1>0&&D1<50&&pT>0.0000000000&&pT<0.5000000000&&xT>0.0000000000&&xT<1.0000000000&& mass>5.000000 && mass<8.000000&&atan(y1_st3/y1_st1)>1.2 && atan(y1_st3/y1_st1) <1.6&& z1_v > -31.5 && z1_v < 145.0&& x1_t*x1_t+(y1_t-1.6)*(y1_t-1.6) <1300.0 && x1_d*x1_d+(y1_d -1.6)*(y1_d-1.6) <93.0&& abs(px1_st1-px1_st3)> 0.415 &&  abs(px1_st1-px1_st3)<0.420 && abs(py1_st1-py1_st3)< 0.01&& abs(pz1_st1-pz1_st3)<0.05&&(chisq1_target-chisq1_dump) >-2.5 && (chisq1_target-chisq1_dump)<77.5&& pz1_st1 > 9 && pz1_st1 < 78 && nHits1 > 13 &&(y1_st1*y1_st3) > 0 && chisq1_dump/chisq1_upstream < 0.195&&atan(y2_st3/y2_st1)>1.2 && atan(y2_st3/y2_st1) <1.6&& z2_v > -31.5 && z2_v < 145.0&& x2_t*x2_t+(y2_t-1.6)*(y2_t-1.6) <1400.0 && x2_d*x2_d+(y2_d-1.6)*(y2_d-1.6) <100.0&& abs(px2_st1-px2_st3)> 0.415 &&  abs(px2_st1-px2_st3)<0.420 && abs(py2_st1-py2_st3)< 0.01&& abs(pz2_st1-pz2_st3)<0.05&&(chisq2_target-chisq2_dump) >-2.5 && (chisq2_target-chisq2_dump)<77.5&&  pz2_st1 > 9 && pz2_st1 < 78 && nHits2 > 13 &&(y2_

Info in <TCanvas::Print>: pdf file plot/acc_corrected_iPT_1.pdf has been created
Info in <TCanvas::Print>: pdf file plot/acc_corrected_iPT_2.pdf has been created
Info in <TCanvas::Print>: pdf file plot/acc_corrected_iPT_3.pdf has been created
Info in <TCanvas::Print>: pdf file plot/acc_corrected_iPT_4.pdf has been created


Figure Details: The left plots are the 2D angular distributions in the lower occupancy (D1<250) regions with different dimuon $pT$ and D1 occupnacy combinations. The right plots are the acceptance corrected result. Right now we are only focusing on the expected statistical accuracy in the lower regions. The $\chi^2/NDF$ in the higher $pT$ needs more investigations.